In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
import requests
import io
import time
from scipy.stats import linregress

# Try to import yahooquery for fast filtering
try:
    from yahooquery import Ticker
except ImportError:
    print("Please install yahooquery: pip install yahooquery")

# ==========================================
# 1. HELPER FUNCTIONS (Your Credit Model)
# ==========================================
def calculate_z_score(info, financials, balance_sheet):
    """Calculates Altman Z-Score"""
    try:
        total_assets = balance_sheet.loc['Total Assets'].iloc[0]
        if 'Total Liabilities Net Minority Interest' in balance_sheet.index:
            total_liab = balance_sheet.loc['Total Liabilities Net Minority Interest'].iloc[0]
        elif 'Total Liabilities' in balance_sheet.index:
            total_liab = balance_sheet.loc['Total Liabilities'].iloc[0]
        else:
            return np.nan
        
        current_assets = balance_sheet.loc['Current Assets'].iloc[0]
        current_liab = balance_sheet.loc['Current Liabilities'].iloc[0]
        working_capital = current_assets - current_liab
        retained_earnings = balance_sheet.loc['Retained Earnings'].iloc[0] if 'Retained Earnings' in balance_sheet.index else 0
        
        if 'Ebit' in financials.index:
            ebit = financials.loc['Ebit'].iloc[0]
        elif 'Operating Income' in financials.index:
            ebit = financials.loc['Operating Income'].iloc[0]
        else:
            return np.nan
            
        market_cap = info.get('marketCap', 0)
        sales = financials.loc['Total Revenue'].iloc[0]

        A = working_capital / total_assets
        B = retained_earnings / total_assets
        C = ebit / total_assets
        D = market_cap / total_liab
        E = sales / total_assets

        z_score = (1.2 * A) + (1.4 * B) + (3.3 * C) + (0.6 * D) + (1.0 * E)
        return round(z_score, 2)
    except:
        return np.nan

def get_margin_trend(financials):
    try:
        limit = min(4, len(financials.columns))
        years = financials.columns[:limit]
        margins = []
        for date in years:
            rev = financials.loc['Total Revenue'][date]
            profit = financials.loc['Gross Profit'][date]
            if rev == 0 or np.isnan(rev): margins.append(0)
            else: margins.append(profit/rev)
            
        if len(margins) < 2: return "N/A"
        slope, _, _, _, _ = linregress(range(len(margins)), margins[::-1])
        if slope > 0.005: return "Improving"
        elif slope < -0.005: return "Deteriorating"
        else: return "Stable"
    except:
        return "N/A"

def get_interest_coverage(financials):
    try:
        if 'Ebit' in financials.index: ebit = financials.loc['Ebit'].iloc[0]
        elif 'Operating Income' in financials.index: ebit = financials.loc['Operating Income'].iloc[0]
        else: return np.nan
        
        if 'Interest Expense' in financials.index: interest = financials.loc['Interest Expense'].iloc[0]
        elif 'Interest Expense Non Operating' in financials.index: interest = financials.loc['Interest Expense Non Operating'].iloc[0]
        else: return 100.0
        
        interest = abs(interest)
        if interest == 0: return 100.0
        return round(ebit / interest, 2)
    except:
        return np.nan

def calculate_roic(financials, balance_sheet):
    try:
        if 'Ebit' in financials.index: ebit = financials.loc['Ebit'].iloc[0]
        elif 'Operating Income' in financials.index: ebit = financials.loc['Operating Income'].iloc[0]
        else: return np.nan
        
        tax_prov = financials.loc['Tax Provision'].iloc[0] if 'Tax Provision' in financials.index else 0
        pre_tax = financials.loc['Pretax Income'].iloc[0] if 'Pretax Income' in financials.index else 0
        tax_rate = max(0.0, min(tax_prov / pre_tax, 0.30)) if pre_tax > 0 else 0.21
        nopat = ebit * (1 - tax_rate)

        equity = balance_sheet.loc['Stockholders Equity'].iloc[0]
        debt = balance_sheet.loc['Total Debt'].iloc[0] if 'Total Debt' in balance_sheet.index else 0
        cash = balance_sheet.loc['Cash And Cash Equivalents'].iloc[0]
        
        invested = equity + debt - cash
        if invested <= 0: return np.nan
        return round((nopat / invested) * 100, 2)
    except:
        return np.nan

# ==========================================
# 2. STEP 1: FETCH UNIVERSE (US + Canada)
# ==========================================
def get_raw_universe():
    print("--- STEP 1: Fetching Raw Stock Lists ---")
    tickers = []
    
    # 1. USA (NASDAQ Traded List)
    try:
        url = "http://www.nasdaqtrader.com/dynamic/symdir/nasdaqtraded.txt"
        df_us = pd.read_csv(url, sep='|')
        df_us = df_us[(df_us['Test Issue'] == 'N') & (df_us['ETF'] == 'N')]
        us_list = df_us['Symbol'].str.replace('.', '-', regex=False).dropna().unique().tolist()
        # Clean junk
        us_list = [t for t in us_list if len(t) < 5 and '$' not in t]
        tickers.extend(us_list)
        print(f"   -> Found {len(us_list)} US candidates.")
    except Exception as e:
        print(f"   Error fetching US list: {e}")

    # 2. CANADA (TSX Composite)
    try:
        url_ca = "https://en.wikipedia.org/wiki/S%26P/TSX_Composite_Index"
        headers = {"User-Agent": "Mozilla/5.0"} 
        r = requests.get(url_ca, headers=headers)
        df_ca = pd.read_html(io.StringIO(r.text))[0]
        
        if 'Symbol' in df_ca.columns:
            ca_list = df_ca['Symbol'].apply(lambda x: x.replace('.', '-') + ".TO").tolist()
            tickers.extend(ca_list)
            print(f"   -> Found {len(ca_list)} Canadian candidates.")
    except Exception as e:
        print(f"   Error fetching Canada list: {e}")

    return list(set(tickers))

# ==========================================
# 3. STEP 2: SPLIT FILTER (The Magic Part)
# ==========================================
def filter_universe_split(ticker_list):
    print(f"\n--- STEP 2: Applying Advanced Filters (YahooQuery) ---")
    
    # --- CONFIGURATION ---
    MIN_PRICE = 5.0
    
    # Regional
    MIN_CAP_US = 300_000_000
    MIN_VOL_US = 1_000_000    # US Volume Floor (1M)
    
    MIN_CAP_CA = 100_000_000
    MIN_VOL_CA = 100_000      # Canada Volume Floor (100k)
    
    # Health
    MIN_CURRENT_RATIO = 1.0
    MIN_OP_MARGIN = 0.001
    
    # Advanced
    MAX_PE = 100.0
    MAX_BETA = 3.0
    
    valid_candidates = []
    chunk_size = 500 
    
    for i in range(0, len(ticker_list), chunk_size):
        chunk = ticker_list[i:i+chunk_size]
        print(f"   Filtering batch {i} - {i+len(chunk)}...", end='\r')
        
        try:
            yq = Ticker(chunk, asynchronous=True)
            data = yq.get_modules("summaryDetail defaultKeyStatistics price financialData")
            
            for symbol in chunk:
                if symbol not in data or isinstance(data[symbol], str): continue
                
                # Extract
                price_mod = data[symbol].get('price', {})
                curr_price = price_mod.get('regularMarketPrice', 0) or 0
                mkt_cap = price_mod.get('marketCap', 0) or 0
                
                summ_mod = data[symbol].get('summaryDetail', {})
                avg_vol = summ_mod.get('averageVolume', 0) or 0
                beta = summ_mod.get('beta', 0) or 0
                pe_ratio = summ_mod.get('trailingPE', 0)
                
                fin_mod = data[symbol].get('financialData', {})
                curr_ratio = fin_mod.get('currentRatio', 0) or 0
                op_margin = fin_mod.get('operatingMargins', 0) or 0
                
                # --- FILTER LOGIC ---
                
                if curr_price < MIN_PRICE: continue
                if curr_ratio < MIN_CURRENT_RATIO: continue
                if op_margin < MIN_OP_MARGIN: continue
                
                # Regional Logic
                is_canada = symbol.endswith('.TO')
                if is_canada:
                    if mkt_cap < MIN_CAP_CA: continue
                    if avg_vol < MIN_VOL_CA: continue
                else:
                    if mkt_cap < MIN_CAP_US: continue
                    if avg_vol < MIN_VOL_US: continue # US 1M Volume Filter

                if MAX_BETA and beta > MAX_BETA: continue
                if MAX_PE and pe_ratio and pe_ratio > MAX_PE: continue

                valid_candidates.append({
                    'Ticker': symbol, 
                    'Price': curr_price,
                    'Market_Cap': mkt_cap,
                    'Region': 'Canada' if is_canada else 'USA',
                    'Current_Ratio': curr_ratio,
                    'Op_Margin': op_margin
                })
                        
        except Exception as e:
            continue
            
    df = pd.DataFrame(valid_candidates)
    print(f"\n   -> Filter complete. Survivors: {len(df)}")
    return df

# ==========================================
# 4. STEP 3: DEEP DIVE (Credit Model)
# ==========================================
def run_credit_model(candidates_df):
    if candidates_df.empty: return None, None, None
    
    print(f"\n--- STEP 3: Deep Analysis (Credit Model + Options Check) ---")
    
    fortress, moonshot, distress = [], [], []
    
    for index, row in candidates_df.iterrows():
        ticker = row['Ticker']
        
        if index % 5 == 0:
            print(f"   Analyzing {index}/{len(candidates_df)}...", end='\r')
            time.sleep(0.2)
            
        try:
            stock = yf.Ticker(ticker)
            
            # 1. Option Check (Reliable)
            try:
                if not stock.options: continue 
            except:
                continue 
            
            # 2. Data
            info = stock.info
            fin = stock.financials
            bs = stock.balance_sheet
            
            if fin.empty or bs.empty: continue
            if 'Financial' in info.get('sector', ''): continue
            
            # 3. Metrics
            z = calculate_z_score(info, fin, bs)
            trend = get_margin_trend(fin)
            int_cov = get_interest_coverage(fin)
            roic = calculate_roic(fin, bs)
            
            item = {
                'Ticker': ticker,
                'Price': row['Price'],
                'Region': row['Region'],
                'Z-Score': z,
                'Margin_Trend': trend,
                'Int_Cov': int_cov,
                'ROIC': roic,
                'Current_Ratio': row['Current_Ratio'],
                'Op_Margin': row['Op_Margin']
            }
            
            # 4. Buckets
            if (z > 2.99) and (trend in ["Improving", "Stable"]) and (int_cov > 4.0) and (roic > 5.0):
                fortress.append(item)
            elif (z < 1.8) or (int_cov < 1.5):
                distress.append(item)
            elif (z < 2.5) and (trend == "Improving"):
                moonshot.append(item)
                
        except:
            continue
            
    return pd.DataFrame(fortress), pd.DataFrame(moonshot), pd.DataFrame(distress)

# --- MAIN EXECUTION ---
# 1. Get Universe
raw_tickers = get_raw_universe()

# 2. Split Filter (US vs Canada)
filtered_df = filter_universe_split(raw_tickers)

# 3. Run Analysis
if not filtered_df.empty:
    fortress_df, moonshot_df, distress_df = run_credit_model(filtered_df)
    
    if not fortress_df.empty:
        print("\n\n--- FORTRESS STOCKS (Top Picks) ---")
        display(fortress_df.sort_values(by='Z-Score', ascending=False).head(10))
    
    if not moonshot_df.empty:
        print("\n--- MOONSHOT STOCKS ---")
        display(moonshot_df.sort_values(by='Z-Score', ascending=False).head(10))

--- STEP 1: Fetching Raw Stock Lists ---
   -> Found 5974 US candidates.

--- STEP 2: Applying Advanced Filters (YahooQuery) ---
   Filtering batch 5500 - 5974...
   -> Filter complete. Survivors: 550

--- STEP 3: Deep Analysis (Credit Model + Options Check) ---
   Analyzing 545/550...

--- FORTRESS STOCKS (Top Picks) ---


,Ticker,Price,Region,Z-Score,Margin_Trend,Int_Cov,ROIC,Current_Ratio,Op_Margin
77,WPM,124.22,USA,206.55,Improving,2569.34,9.30,8.089,0.66542
69,APP,714.23,USA,32.58,Improving,5.89,47.98,3.250,0.76795
44,FAST,41.56,USA,30.85,Stable,206.85,29.96,4.259,0.20696
11,MNST,77.31,USA,29.27,Stable,100.00,30.51,3.185,0.30738
86,ADMA,19.28,USA,21.37,Improving,9.98,42.37,7.128,0.38005
58,TSEM,121.61,USA,20.13,Stable,45.93,6.88,6.610,0.12783
110,ODFL,159.49,USA,19.18,Stable,7283.01,28.02,1.204,0.25655
14,CPRX,24.35,USA,17.02,Improving,100.00,69.34,6.622,0.44657
30,VEEV,224.65,USA,16.85,Stable,100.00,11.21,7.527,0.29690
92,LRCX,178.07,USA,15.86,Improving,33.11,66.71,2.207,0.34354



--- MOONSHOT STOCKS ---


,Ticker,Price,Region,Z-Score,Margin_Trend,Int_Cov,ROIC,Current_Ratio,Op_Margin
16,MBC,11.27,USA,2.47,Improving,3.37,8.28,1.865,0.08499
10,EGO,37.18,USA,2.47,Improving,20.69,7.33,2.794,0.39688
23,OC,113.74,USA,2.43,Improving,8.02,11.52,1.396,0.18070
5,HAYW,15.94,USA,2.42,Improving,3.16,8.04,3.184,0.16915
4,FLNC,20.21,USA,2.36,Improving,NaN,-28.24,1.506,0.04707
9,FTI,44.65,USA,2.35,Improving,10.09,23.98,1.130,0.15299
1,CXM,7.72,USA,2.34,Improving,100.00,4.65,1.657,0.05515
22,KBR,40.22,USA,2.34,Improving,3.81,10.02,1.177,0.06266
14,VAL,49.51,USA,2.33,Improving,4.28,11.95,1.871,0.21135
15,CCK,102.90,USA,2.32,Improving,3.31,13.64,1.155,0.13991
